# Claudia Pinzón

## Taller No. 5

In [1]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

# Punto 1: Pre-Procesamiento

- `[9 pts]` Leer el archivo `bob_esponja.csv` usando `pandas`
- `[9 pts]` Crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [2]:
# Leer archivo

doc = pd.read_csv("C:/Users/eliza/OneDrive/Desktop/Claus/NLP/bob_esponja.csv")
print(doc)

                          Personaje  \
0  Bob Esponja Pantalones Cuadrados   
1                 Patricio Estrella   
2              Calamardo Tentáculos   
3                  Arenita Mejillas   
4               Eugenio H. Cangrejo   
5               Sheldon J. Plankton   
6                    Karen Plankton   
7                  Perlita Cangrejo   
8                         Sra. Puff   
9                   Gary el Caracol   

                                         Descripción  
0  Bob Esponja es una esponja de mar con forma re...  
1  Es una estrella de mar de color rosa, el mejor...  
2  Calamardo Tentáculos es un personaje principal...  
3  Arenita Mejillas (Sandy Cheeks en inglés) en u...  
4  Don Eugenio H. Cangrejo (normalmente llamado S...  
5  Sheldon J. Plankton, o simplemente Plankton, e...  
6  Karen Plankton es uno de los dos principales a...  
7  Perla "Perlita" Cangrejo es un personaje princ...  
8  La Señora Puff es un personaje principal de Bo...  
9  Gary el Caracol e

In [3]:
#Nueva columna con modificaciones

def pre_procesado(texto):
    texto = texto.lower()                                                         #pasar a minúsculas
    texto = re.sub(r"[\W\d_´]+", " ", texto)                                      #eliminar caracteres
    texto = [palabra for palabra in texto.split() if palabra not in stopwords_sp] #quitar palabras vacías
    texto = [spanishStemmer.stem(texto) for texto in texto]                       #stemming de las palabras
    texto = [texto.lower() for texto in texto if len(texto) > 2 and texto.isalpha()] #quitar palabras de menos de 2 caracteres
    texto = " ".join(texto)
    return (texto)

doc['pre_procesado']=doc['Descripción'].apply(lambda texto: pre_procesado(texto))
doc

,Personaje,Descripción,pre_procesado
0,Bob Esponja Pantalones Cuadrados,Bob Esponja es una esponja de mar con forma re...,bob esponj esponj mar form rectangul color ama...
1,Patricio Estrella,"Es una estrella de mar de color rosa, el mejor...",estrell mar color ros mejor amig bob esponj ju...
2,Calamardo Tentáculos,Calamardo Tentáculos es un personaje principal...,calamard tentacul personaj principal bob espon...
3,Arenita Mejillas,Arenita Mejillas (Sandy Cheeks en inglés) en u...,arenit mejill sandy cheeks ingles personaj pri...
4,Eugenio H. Cangrejo,Don Eugenio H. Cangrejo (normalmente llamado S...,don eugeni cangrej normal llam señor cangrej e...
5,Sheldon J. Plankton,"Sheldon J. Plankton, o simplemente Plankton, e...",sheldon plankton simplement plankton dos antag...
6,Karen Plankton,Karen Plankton es uno de los dos principales a...,kar plankton dos principal antagon bob esponj ...
7,Perlita Cangrejo,"Perla ""Perlita"" Cangrejo es un personaje princ...",perl perlit cangrej personaj principal bob esp...
8,Sra. Puff,La Señora Puff es un personaje principal de Bo...,señor puff personaj principal bob esponj maest...
9,Gary el Caracol,Gary el Caracol es un personaje principal de B...,gary caracol personaj principal bob esponj que...


# Punto 2: TF-IDF

- `[16 pts]` Crear la matriz TF-IDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(doc['pre_procesado'].values)

tfidf_matrix = pd.DataFrame(tfidf.toarray(), columns=tfidf_vect.get_feature_names())
tfidf_matrix.index = doc.index

tfidf_matrix = tfidf_matrix.T.round(3)
tfidf_matrix.rename(columns={0:'Bob Esponja',1:'Patricio',2:'Calamardo',3:'Arenita',4:'Don Cangrejo',
                             5:'Plankton',6:'Karen',7:'Perlita',8:'Sñ Puff',9:'Gary'}, inplace=True)
tfidf_matrix

,Bob Esponja,Patricio,Calamardo,Arenita,Don Cangrejo,Plankton,Karen,Perlita,Sñ Puff,Gary
abaj,0.058,0.00,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000
aborrec,0.000,0.00,0.091,0.000,0.000,0.0,0.000,0.0,0.000,0.000
acab,0.000,0.00,0.091,0.000,0.000,0.0,0.000,0.0,0.000,0.000
accident,0.049,0.00,0.000,0.000,0.045,0.0,0.000,0.0,0.000,0.000
acept,0.000,0.00,0.000,0.000,0.000,0.0,0.064,0.0,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...
viv,0.000,0.05,0.060,0.108,0.070,0.0,0.000,0.0,0.000,0.000
vol,0.058,0.00,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000
voz,0.197,0.00,0.000,0.139,0.000,0.0,0.000,0.0,0.000,0.000
vuelv,0.173,0.00,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000


# Punto 3: Distancia del coseno
- `[12 pts]` Calcular la distancia del coseno entre cada una de los personajes
- `[2 pts]` ¿Cuáles son los personajes más parecidos?
- `[2 pts]` ¿Cuáles son los personajes más diferentes?

In [5]:
#Convertir matriz en valores para calcular distancia

from sklearn.metrics.pairwise import euclidean_distances

#distancia euclidiana

dist_euc = euclidean_distances(tfidf_matrix.T.values)
dist_euc = pd.DataFrame(dist_euc, columns = tfidf_matrix.columns, index=tfidf_matrix.columns)

print(dist_euc)

              Bob Esponja  Patricio  Calamardo   Arenita  Don Cangrejo  \
Bob Esponja      0.000000  1.288340   1.354368  1.293962      1.316096   
Patricio         1.288340  0.000000   1.254619  1.295178      1.286424   
Calamardo        1.354368  1.254619   0.000000  1.355360      1.287268   
Arenita          1.293962  1.295178   1.355360  0.000000      1.346273   
Don Cangrejo     1.316096  1.286424   1.287268  1.346273      0.000000   
Plankton         1.328056  1.391621   1.352579  1.390396      1.171983   
Karen            1.369766  1.361863   1.377084  1.376924      1.333965   
Perlita          1.330836  1.333937   1.390434  1.387666      1.262891   
Sñ Puff          1.309910  1.379223   1.391162  1.395014      1.337948   
Gary             1.359776  1.336357   1.304595  1.342611      1.356311   

              Plankton     Karen   Perlita   Sñ Puff      Gary  
Bob Esponja   1.328056  1.369766  1.330836  1.309910  1.359776  
Patricio      1.391621  1.361863  1.333937  1.379223  1

In [6]:
# distancia del coseno entre personajes
from sklearn.metrics.pairwise import cosine_distances

dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_euc, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos

,Bob Esponja,Patricio,Calamardo,Arenita,Don Cangrejo,Plankton,Karen,Perlita,Sñ Puff,Gary
Bob Esponja,0.000000,1.288340,1.354368,1.293962,1.316096,1.328056,1.369766,1.330836,1.309910,1.359776
Patricio,1.288340,0.000000,1.254619,1.295178,1.286424,1.391621,1.361863,1.333937,1.379223,1.336357
Calamardo,1.354368,1.254619,0.000000,1.355360,1.287268,1.352579,1.377084,1.390434,1.391162,1.304595
Arenita,1.293962,1.295178,1.355360,0.000000,1.346273,1.390396,1.376924,1.387666,1.395014,1.342611
Don Cangrejo,1.316096,1.286424,1.287268,1.346273,0.000000,1.171983,1.333965,1.262891,1.337948,1.356311
Plankton,1.328056,1.391621,1.352579,1.390396,1.171983,0.000000,1.258256,1.357786,1.404833,1.392228
Karen,1.369766,1.361863,1.377084,1.376924,1.333965,1.258256,0.000000,1.390313,1.403868,1.389705
Perlita,1.330836,1.333937,1.390434,1.387666,1.262891,1.357786,1.390313,0.000000,1.322880,1.393592
Sñ Puff,1.309910,1.379223,1.391162,1.395014,1.337948,1.404833,1.403868,1.322880,0.000000,1.389927
Gary,1.359776,1.336357,1.304595,1.342611,1.356311,1.392228,1.389705,1.393592,1.389927,0.000000


# ¿Cuáles son los personajes más parecidos?

In [ ]:
#Bob Esponja y Patricio 
#Patricio y Calamardo
#Calamardo y Arenita
#Arenita y Bob Esponja
#Don Cangrejo y Plankton
#Karen y Plankton
#Perlita y Don Cangrejo
#Sñ Puff y Bob Esponja
#Gary y Patricio

# ¿Cuáles son los personajes más diferentes?

In [ ]:
#Bob Esponja y Karen
#Patricio y Plankton
#Calamardo y Sñ Puff
#Arenita y Sñ Puff
#Don Cangrejo y Gary
#Plankton y Sñ Puff
#Karen y Sñ Puff
#Perlita y Gary